In [1]:
from sys import platform

from sympy.physics.units import pascal

if platform == "linux" or platform == "linux2":
    !pip install -U albumentations
    !pip install xmltodict
    !pip install imagesize
    !pip install nvidia-dali-cuda120
    !pip install ultralytics

    !git clone https://github.com/HubertDomaros/ai-capstone-proj --branch dev
    import os
    os.chdir(os.path.join(os.getcwd(), 'ai-capstone-proj'))
    os.getcwd()

    from src.nvidia_dali_resizer import resize_with_dali

In [12]:
import src.utils
from src.std_imports import os, pd

from src import annotations_processing as ap
from src import image_processing as ip
from src import constants as c
from src import utils

if platform == "linux" or platform == "linux2":
    pascal_dataset_dir = '/kaggle/input/codebrim-original/original_dataset'
    yolo_dataset_dir = '/kaggle/working/datasets'

else:
    pascal_dataset_dir = os.path.join(os.getcwd(), r'kaggle\input\codebrim-original\original_dataset')
    yolo_dataset_dir = os.path.join(os.getcwd(), r'datasets')

pascal_img_dir = os.path.join(pascal_dataset_dir, 'images')
pascal_annotations_dir = os.path.join(pascal_dataset_dir, 'annotations')
yolo_images_dir = os.path.join(yolo_dataset_dir, 'images')
yolo_labels_dir = os.path.join(yolo_dataset_dir, 'labels')

pascal_df: pd.DataFrame = ap.xml_annotations_to_dataframe(pascal_annotations_dir)

pascal_df

<>:11: SyntaxWarning: invalid escape sequence '\d'
<>:11: SyntaxWarning: invalid escape sequence '\d'
C:\Users\Hubert\AppData\Local\Temp\ipykernel_30076\3473198193.py:11: SyntaxWarning: invalid escape sequence '\d'
  yolo_dataset_dir = '/kaggle/working\datasets'


,img,img_width,img_height,xmin,ymin,xmax,ymax,Background,Crack,Spallation,Efflorescence,ExposedBars,CorrosionStain
0,image_0000005.jpg,1904,2856,661,472,992,1857,0,0,0,1,0,1
1,image_0000005.jpg,1904,2856,1507,505,1904,2856,0,0,0,1,0,1
2,image_0000021.jpg,1904,2856,253,8,335,2108,0,0,0,1,0,0
3,image_0000028.jpg,1752,2632,1,1,907,1043,0,0,0,1,0,1
4,image_0000028.jpg,1752,2632,1173,1,1747,1077,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5256,image_0001599.jpg,4608,3456,2613,1,2776,408,0,0,1,0,1,1
5257,image_0001599.jpg,4608,3456,1921,1,3105,800,0,0,1,0,1,1
5258,image_0001599.jpg,4608,3456,226,1,647,2710,0,0,0,0,0,1
5259,image_0001599.jpg,4608,3456,1002,1097,2931,3456,0,0,0,1,0,1


In [3]:
pascal_df = ap.fill_missing_imgs_in_df(pascal_img_dir, pascal_df)

In [4]:
pascal_df.sort_values(by=[c.IMG], ascending=True, inplace=True)
pascal_df

,img,img_width,img_height,xmin,ymin,xmax,ymax,Background,Crack,Spallation,Efflorescence,ExposedBars,CorrosionStain
5261,image_0000001.jpg,1712,2572,0,0,0,0,1,0,0,0,0,0
5262,image_0000002.jpg,1732,2596,0,0,0,0,1,0,0,0,0,0
5263,image_0000003.jpg,1732,2596,0,0,0,0,1,0,0,0,0,0
5264,image_0000004.jpg,1688,2532,0,0,0,0,1,0,0,0,0,0
0,image_0000005.jpg,1904,2856,661,472,992,1857,0,0,0,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5257,image_0001599.jpg,4608,3456,1921,1,3105,800,0,0,1,0,1,1
5258,image_0001599.jpg,4608,3456,226,1,647,2710,0,0,0,0,0,1
5259,image_0001599.jpg,4608,3456,1002,1097,2931,3456,0,0,0,1,0,1
5253,image_0001599.jpg,4608,3456,3339,734,3736,2826,0,0,0,0,0,1


In [6]:
yolo_df = ap.pascal_df_to_yolo(pascal_df)
yolo_df[yolo_df[c.yolo_cols_list[3:7]].gt(1).any(axis=1)]

,img,img_width,img_height,Background,Crack,Spallation,Efflorescence,ExposedBars,CorrosionStain,multihot_encoding_class,x_center,y_center,bbox_width,bbox_height


In [16]:
ap.save_yolo_annotations(yolo_df, yolo_labels_dir)

Saved 1590 annotations to D:\0-Code\PG\2_sem\0_Dyplom\ai-capstone-proj\datasets\labels


In [9]:
train_test_val = src.utils.train_test_val_image_split(yolo_df, test_size=0.2, val_size=0.1)
train_test_val['train'][0]

'image_0000001.jpg'

In [10]:
if platform == "linux" or platform == "linux2":
    resize_with_dali(pascal_img_dir, yolo_images_dir)
# elif not os.path.exists(yolo_images_dir):
#     os.makedirs(yolo_images_dir, exist_ok=True)
#     ip.resize_images_pytorch(input_dir=pascal_img_dir, output_dir=yolo_images_dir,
#                             target_width=640, target_height=640, delete_output_dir=True)

In [19]:
if platform == "linux" or platform == "linux2":
    utils.put_imgs_in_folders(train_test_val, yolo_images_dir, yolo_images_dir)
else:
    utils.put_imgs_in_folders(train_test_val, r'images\images_resized', yolo_images_dir)

utils.put_labels_in_folders(train_test_val, yolo_labels_dir, yolo_labels_dir)

Copying images to D:\0-Code\PG\2_sem\0_Dyplom\ai-capstone-proj\datasets\images\train finished!
Copying images to D:\0-Code\PG\2_sem\0_Dyplom\ai-capstone-proj\datasets\images\test finished!
Copying images to D:\0-Code\PG\2_sem\0_Dyplom\ai-capstone-proj\datasets\images\val finished!
Copying images to D:\0-Code\PG\2_sem\0_Dyplom\ai-capstone-proj\datasets\labels\train finished!
Copying images to D:\0-Code\PG\2_sem\0_Dyplom\ai-capstone-proj\datasets\labels\test finished!
Copying images to D:\0-Code\PG\2_sem\0_Dyplom\ai-capstone-proj\datasets\labels\val finished!


In [ ]:
from ultralytics import YOLO
import torch

torch.cuda.is_available()

model = YOLO("yolo11n.pt")
results = model.train(data="yolo_windows.yaml", epochs=100, imgsz=640, device='0', batch=8)